In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re 
import string


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import spacy
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TabTokenizer
tokenizer = TabTokenizer()
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Training 

In [ ]:
pip install torch

In [ ]:
pip install transformers

In [ ]:
from transformers import AutoModel

In [ ]:
import pandas as pd
import numpy as np
import requests
from transformers import AutoTokenizer
from transformers import pipeline
import datasets
from datasets import Dataset
!pip3 install deberta
#pip install transformers

In [ ]:
pip install datasets

In [ ]:
import torch
from transformers import DebertaTokenizer, DebertaModel

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

In [ ]:
def clean_text(text):
#replace the html characters with " "
    text=re.sub('<.*?>', ' ', text)  
#remove the punctuations
    text = text.translate(str.maketrans(' ',' ',string.punctuation))
#consider only alphabets and numerics
    text = re.sub('[^a-zA-Z]',' ',text)  
#replace newline with space
    text = re.sub("\n"," ",text)
#convert to lower case
    text = text.lower()
#split and join the words
    text=' '.join(text.split())
    return text

In [ ]:
# url = "https://raw.githubusercontent.com/lobnadoma/data/main/comments_to_score.csv"
# df = pd.read_csv(url)
cts = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv")
validation = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv")
sample = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv")

In [ ]:
df2 = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv")
df3 = df2[['id','comment_text','toxic','severe_toxic']]

In [ ]:
df2.head()

In [ ]:
df2[(df2['insult']==1) & (df2['toxic']==1)]

## Data Cleaning

In [ ]:
df3['comment_text'] = df3['comment_text'].replace(regex=['<.*?>'],value=' ')
df3['comment_text'] = df3['comment_text'].replace(regex=['[^a-zA-Z]'],value=' ')
df3['comment_text'] = df3['comment_text'].replace(regex=['\n'],value=' ')

In [ ]:
#df3['comment_text'] = df3['comment_text'].apply(lambda x: ''.join([w for w in clean_text(x)]))
cts['text'] = cts['text'].apply(lambda x: ''.join([w for w in clean_text(x)]))


In [ ]:
df2['other'] = df2['obscene']+df2['threat']+df2['insult']+df2['identity_hate']

In [ ]:
df2[((df2['toxic']==0) & (df2['other']>0))]

In [ ]:
df3['toxic'] = df3['toxic'] + df3['severe_toxic']

In [ ]:
df3[df3['toxic']>0]



In [ ]:
df4 = df3[['toxic','comment_text']]
df4.columns = ['label','text']

In [ ]:
df4 = df4.sample(frac=0.1, replace=False, random_state=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df4['text'], df4['label'], test_size=0.33, random_state=42)

In [ ]:
d = {'text': X_train.values.tolist(), 'label': y_train.values.tolist()}
train = pd.DataFrame(data=d)
d = {'text': X_test.values.tolist(), 'label': y_test.values.tolist()}
test = pd.DataFrame(data=d)
train['label'] = train.label.astype("category").cat.codes
test['label'] = test.label.astype("category").cat.codes

In [ ]:
train_d = Dataset.from_pandas(train)
test_d = Dataset.from_pandas(test)

In [ ]:
test_d

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"],  padding="max_length", truncation=True)


tokenized_train = train_d.map(tokenize_function, batched=True)
tokenized_test= test_d.map(tokenize_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification
#DebertaModel
model = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-base', problem_type="multi_label_classification")
#model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=7)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments("test_trainer")

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args, 
    train_dataset=tokenized_train, 
    eval_dataset=tokenized_test
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained('deberta-model')

In [ ]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)
clf = LogisticRegression(max_iter = 200).fit(X_train_tfidf, y_train)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(clf.score(X_test_tfidf, y_test)))

In [ ]:
y_pred = clf.predict_proba(vectorizer.transform(cts['text']))

In [ ]:
clf.classes_

In [ ]:
probs = pd.DataFrame(y_pred, columns = ['not','toxic','severe'])

In [ ]:
probs['combined'] = probs['toxic'] + probs['severe']

In [ ]:
cts2 = cts.copy()
cts2['score'] = probs['combined']

In [ ]:
cts2 = cts2[['comment_id','score']]

In [ ]:
cts2['score'] = cts2['score'].round(3)

In [ ]:
cts2.head()

In [ ]:
cts2.to_csv('submission.csv', index=False)

## Transformer Model

In [ ]:
pip install wandb --upgrade

In [ ]:
pip install transformers

In [ ]:
import transformers
from transformers import AutoModel, AutoTokenizer
transformers.__version__

In [ ]:
model_path_or_name = '../input/transformers/roberta-base'
model     = AutoModel.from_pretrained(model_path_or_name)
tokenizer = AutoTokenizer.from_pretrained(model_path_or_name)

In [ ]:
import pandas as pd
import numpy as np
import requests
from transformers import AutoTokenizer
from transformers import pipeline
# import datasets
# from datasets import Dataset
!pip3 install deberta
#pip install transformers

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh |  bash
!apt-get install -y --allow-unauthenticated git-lfs

In [ ]:
!git lfs install
!git clone https://huggingface.co/GroNLP/hateBERT
# if you want to clone without large files – just their pointers
# prepend your git clone with the following env var:
!GIT_LFS_SKIP_SMUDGE=1

In [ ]:
def setup_tensorflow_1_13():
    
    # Install `tensorflow-gpu==1.13.1` from pre-downloaded wheels
    PATH_TO_TF_WHEELS = '/kaggle/input/tensorflow1131-offline-bert/tensorflow_gpu_1_13_1_with_deps_whl/tensorflow_gpu_1_13_1_with_deps_whl'
    # yes, mixing up Python code and bash is ugly. But it's handy 
    !pip install --no-deps $PATH_TO_TF_WHEELS/*.whl


In [ ]:
ruddit = pd.read_csv("../input/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv")
print(ruddit.shape)

In [ ]:
ruddit.head()

In [ ]:
ruddit = ruddit[['txt', 'offensiveness_score']].rename(columns={'txt': 'text',
                                                                'offensiveness_score':'y'})

In [ ]:
#Rescale values to range between 0 and 1
ruddit['y'] = (ruddit['y'] - ruddit.y.min()) / (ruddit.y.max() - ruddit.y.min()) 
ruddit.y.hist()

In [ ]:
pip install pytorch_transformers

In [ ]:
import torch
import pytorch_transformers
import datasets
from datasets import Dataset

In [ ]:
from transformers import AutoConfig
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import AutoTokenizer,AutoModelForSequenceClassification
config = AutoConfig.from_pretrained(f'../input/hatebert/')
tokenizer = AutoTokenizer.from_pretrained(f'../input/hatebert/')
model = AutoModelForSequenceClassification.from_pretrained(f'../input/hatebert/', num_labels = 3)

In [ ]:
df4 = df3[['toxic','comment_text']]
df4.columns = ['label','text']

In [ ]:
df4 = df4.sample(frac=0.25, replace=False, random_state=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df4['text'], df4['label'], test_size=0.33, random_state=42)

In [ ]:
d = {'text': X_train.values.tolist(), 'label': y_train.values.tolist()}
train = pd.DataFrame(data=d)

In [ ]:
d = {'text': X_train.values.tolist(), 'label': y_train.values.tolist()}
train = pd.DataFrame(data=d)
d = {'text': X_test.values.tolist(), 'label': y_test.values.tolist()}
test = pd.DataFrame(data=d)
train['label'] = train.label.astype("category").cat.codes
test['label'] = test.label.astype("category").cat.codes

In [ ]:
train_d['label'][0]

In [ ]:
train_d['text'][0]

In [ ]:
train_d = Dataset.from_pandas(train)
test_d = Dataset.from_pandas(test)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train_d.map(tokenize_function, batched=True)
tokenized_test= test_d.map(tokenize_function, batched=True)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments("test_trainer")

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args, 
    train_dataset=tokenized_train, 
    eval_dataset=tokenized_test
)

In [ ]:
tokenized_train['input_ids'][0]

In [ ]:
trainer.train()

In [ ]:
!git lfs install
!git clone https://huggingface.co/GroNLP/hateBERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# New Section

In [ ]:
import os
import gc
import copy
import time
import random
import string

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AdamW

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")


In [ ]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

In [ ]:
def id_generator(size=12, chars=string.ascii_lowercase + string.digits):
    return ''.join(random.SystemRandom().choice(chars) for _ in range(size))

HASH_NAME = id_generator(size=12)
print(HASH_NAME)

In [ ]:
CONFIG = {"seed": 42,
          "epochs": 3,
          "model_name": "GroNLP/hateBERT",
          "train_batch_size": 16,
          "valid_batch_size": 32,
          'gradient_accumulation_steps' : 2,
          "max_length": 256,
          "learning_rate": 1e-5,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-6,
          "T_max": 500,
          "weight_decay": 1e-6,
          "n_fold": 5,
          "n_accumulate": 1,
          "num_classes": 1,
          "margin": 0.5,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          "hash_name": HASH_NAME
          }

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])
CONFIG['group'] = f'{HASH_NAME}-Baseline'


In [ ]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [ ]:
df = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")
df.head()

In [ ]:
skf = StratifiedKFold(n_splits=CONFIG['n_fold'], shuffle=True, random_state=CONFIG['seed'])

for fold, ( _, val_) in enumerate(skf.split(X=df, y=df.worker)):
    df.loc[val_ , "kfold"] = int(fold)
    
df["kfold"] = df["kfold"].astype(int)
df.head()

## Dataset Class

In [ ]:
class Dataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_lenth
        self.text = df['text'].values
        self.toxicity = df['toxicity'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        toxicity = self.toxicity[index]
        inputs_more_toxic = self.tokenizer.encode_plus(
                                more_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        inputs_less_toxic = self.tokenizer.encode_plus(
                                less_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        target = 1
        
        more_toxic_ids = inputs_more_toxic['input_ids']
        more_toxic_mask = inputs_more_toxic['attention_mask']
        
        less_toxic_ids = inputs_less_toxic['input_ids']
        less_toxic_mask = inputs_less_toxic['attention_mask']
        
        
        return {
            'more_toxic_ids': torch.tensor(more_toxic_ids, dtype=torch.long),
            'more_toxic_mask': torch.tensor(more_toxic_mask, dtype=torch.long),
            'less_toxic_ids': torch.tensor(less_toxic_ids, dtype=torch.long),
            'less_toxic_mask': torch.tensor(less_toxic_mask, dtype=torch.long),
            'target': torch.tensor(target, dtype=torch.long)
        }
        

In [ ]:

class Dataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_lenth
        self.more_toxic = df['more_toxic'].values
        self.less_toxic = df['less_toxic'].values
    
    def __getitem__(self, index):
        more_toxic = self.more_toxic[index]
        less_toxic = self.less_toxic[index]
        
        
    def __getitem__(self, index):
        more_toxic = self.more_toxic[index]
        less_toxic = self.less_toxic[index]
        inputs_more_toxic = self.tokenizer.encode_plus(
                                more_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        inputs_less_toxic = self.tokenizer.encode_plus(
                                less_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        target = 1
        
        more_toxic_ids = inputs_more_toxic['input_ids']
        more_toxic_mask = inputs_more_toxic['attention_mask']
        
        less_toxic_ids = inputs_less_toxic['input_ids']
        less_toxic_mask = inputs_less_toxic['attention_mask']
        
        
        return {
            'more_toxic_ids': torch.tensor(more_toxic_ids, dtype=torch.long),
            'more_toxic_mask': torch.tensor(more_toxic_mask, dtype=torch.long),
            'less_toxic_ids': torch.tensor(less_toxic_ids, dtype=torch.long),
            'less_toxic_mask': torch.tensor(less_toxic_mask, dtype=torch.long),
            'target': torch.tensor(target, dtype=torch.long)
        }
    def __len__(self):
        return len(self.df)
    
    class JigsawModel(nn.Module):
    def __init__(self, model_name):
        super(JigsawModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(768, CONFIG['num_classes'])
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.drop(out[1])
        outputs = self.fc(out)
        return outputs
    
    
    def criterion(outputs1, outputs2, targets):
    return nn.MarginRankingLoss(margin=CONFIG['margin'])(outputs1, outputs2, targets)


In [ ]:
class JigsawDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.more_toxic = df['more_toxic'].values
        self.less_toxic = df['less_toxic'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        more_toxic = self.more_toxic[index]
        less_toxic = self.less_toxic[index]
        inputs_more_toxic = self.tokenizer.encode_plus(
                                more_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        inputs_less_toxic = self.tokenizer.encode_plus(
                                less_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        target = 1
        
        more_toxic_ids = inputs_more_toxic['input_ids']
        more_toxic_mask = inputs_more_toxic['attention_mask']
        
        less_toxic_ids = inputs_less_toxic['input_ids']
        less_toxic_mask = inputs_less_toxic['attention_mask']
        
        
        return {
            'more_toxic_ids': torch.tensor(more_toxic_ids, dtype=torch.long),
            'more_toxic_mask': torch.tensor(more_toxic_mask, dtype=torch.long),
            'less_toxic_ids': torch.tensor(less_toxic_ids, dtype=torch.long),
            'less_toxic_mask': torch.tensor(less_toxic_mask, dtype=torch.long),
            'target': torch.tensor(target, dtype=torch.long)
        }

## Model Class

In [ ]:
class JigsawModel(nn.Module):
    def __init__(self, model_name):
        super(JigsawModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(768, CONFIG['num_classes'])
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.drop(out[1])
        outputs = self.fc(out)
        return outputs

## Loss Function


In [ ]:
def criterion(outputs1, outputs2, targets):
    return nn.MarginRankingLoss(margin=CONFIG['margin'])(outputs1, outputs2, targets)


## Training function

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        more_toxic_ids = data['more_toxic_ids'].to(device, dtype = torch.long)
        more_toxic_mask = data['more_toxic_mask'].to(device, dtype = torch.long)
        less_toxic_ids = data['less_toxic_ids'].to(device, dtype = torch.long)
        less_toxic_mask = data['less_toxic_mask'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.long)
        
        batch_size = more_toxic_ids.size(0)

        more_toxic_outputs = model(more_toxic_ids, more_toxic_mask)
        less_toxic_outputs = model(less_toxic_ids, less_toxic_mask)
        
        loss = criterion(more_toxic_outputs, less_toxic_outputs, targets)
        loss = loss / CONFIG['n_accumulate']
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss


In [ ]:
### Modified
def train_all_epoch(model, optimizer, scheduler, dataloader, device, epoch, num_epochs):
    train_loss = []
    
    for epoch in range(1, num_epochs + 1): 
        model.train()

        dataset_size = 0
        running_loss = 0.0

        bar = tqdm(enumerate(dataloader), total=len(dataloader))
        for step, data in bar:
            more_toxic_ids = data['more_toxic_ids'].to(device, dtype = torch.long)
            more_toxic_mask = data['more_toxic_mask'].to(device, dtype = torch.long)
            less_toxic_ids = data['less_toxic_ids'].to(device, dtype = torch.long)
            less_toxic_mask = data['less_toxic_mask'].to(device, dtype = torch.long)
            targets = data['target'].to(device, dtype=torch.long)

            batch_size = more_toxic_ids.size(0)

            more_toxic_outputs = model(more_toxic_ids, more_toxic_mask)
            less_toxic_outputs = model(less_toxic_ids, less_toxic_mask)

            loss = criterion(more_toxic_outputs, less_toxic_outputs, targets)
            loss = loss / CONFIG['n_accumulate']
            loss.backward()

            if (step + 1) % CONFIG['n_accumulate'] == 0:
                optimizer.step()

                # zero the parameter gradients
                optimizer.zero_grad()

                if scheduler is not None:
                    scheduler.step()

            running_loss += (loss.item() * batch_size)
            dataset_size += batch_size

            epoch_loss = running_loss / dataset_size

            bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                            LR=optimizer.param_groups[0]['lr'])
        gc.collect()

        train_loss.append(epoch_loss)



# def run_training(model, optimizer, scheduler, device, num_epochs, fold):
#     # To automatically log gradients
#     wandb.watch(model, log_freq=100)
    
#     if torch.cuda.is_available():
#         print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
#     start = time.time()
#     best_model_wts = copy.deepcopy(model.state_dict())
#     best_epoch_loss = np.inf
#     history = defaultdict(list)
    
#     for epoch in range(1, num_epochs + 1): 
#         gc.collect()
#         train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
#                                            dataloader=train_loader, 
#                                            device=CONFIG['device'], epoch=epoch)
        
#         val_epoch_loss = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
#                                          epoch=epoch)
    
#         history['Train Loss'].append(train_epoch_loss)
#         history['Valid Loss'].append(val_epoch_loss)
        
#         # Log the metrics
#         wandb.log({"Train Loss": train_epoch_loss})
#         wandb.log({"Valid Loss": val_epoch_loss})
        
#         # deep copy the model
#         if val_epoch_loss <= best_epoch_loss:
#             print(f"{b_}Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
#             best_epoch_loss = val_epoch_loss
#             run.summary["Best Loss"] = best_epoch_loss
#             best_model_wts = copy.deepcopy(model.state_dict())
#             PATH = f"Loss-Fold-{fold}.bin"
#             torch.save(model.state_dict(), PATH)
#             # Save a model file from the current directory
#             print(f"Model Saved{sr_}")
            
#         print()
    
#     end = time.time()
#     time_elapsed = end - start
#     print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
#         time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
#     print("Best Loss: {:.4f}".format(best_epoch_loss))
    
#     # load best model weights
#     model.load_state_dict(best_model_wts)
    
#     return model, history



## Validation Function

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        more_toxic_ids = data['more_toxic_ids'].to(device, dtype = torch.long)
        more_toxic_mask = data['more_toxic_mask'].to(device, dtype = torch.long)
        less_toxic_ids = data['less_toxic_ids'].to(device, dtype = torch.long)
        less_toxic_mask = data['less_toxic_mask'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.long)
        
        batch_size = more_toxic_ids.size(0)

        more_toxic_outputs = model(more_toxic_ids, more_toxic_mask)
        less_toxic_outputs = model(less_toxic_ids, less_toxic_mask)
        
        loss = criterion(more_toxic_outputs, less_toxic_outputs, targets)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    
    gc.collect()
    
    return epoch_loss


## Run Training

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs, fold):
    # To automatically log gradients
    wandb.watch(model, log_freq=100)
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        
        # Log the metrics
        wandb.log({"Train Loss": train_epoch_loss})
        wandb.log({"Valid Loss": val_epoch_loss})
        
        # deep copy the model
        if val_epoch_loss <= best_epoch_loss:
            print(f"{b_}Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss
            run.summary["Best Loss"] = best_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"Loss-Fold-{fold}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved{sr_}")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history


In [ ]:
def prepare_loaders(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_dataset = JigsawDataset(df_train, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])
    valid_dataset = JigsawDataset(df_valid, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

In [ ]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

## Start Training

In [ ]:
for fold in range(0, CONFIG['n_fold']):
    print(f"{y_}====== Fold: {fold} ======{sr_}")
    run = wandb.init(project='Jigsaw', 
                     config=CONFIG,
                     job_type='Train',
                     group=CONFIG['group'],
                     tags=['hateBERT', f'{HASH_NAME}', 'margin-loss'],
                     name=f'{HASH_NAME}-fold-{fold}',
                     anonymous='must')
    
    # Create Dataloaders
    train_loader, valid_loader = prepare_loaders(fold=fold)
    
    model = JigsawModel(CONFIG['model_name'])
    model.to(CONFIG['device'])
    
    # Define Optimizer and Scheduler
    optimizer = AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
    scheduler = fetch_scheduler(optimizer)
    
    model, history = run_training(model, optimizer, scheduler,
                                  device=CONFIG['device'],
                                  num_epochs=CONFIG['epochs'],
                                  fold=fold)
    
    run.finish()
    
    del model, history, train_loader, valid_loader
    _ = gc.collect()
    print()